In [24]:
import pandas as pd 
import numpy as np 

In [25]:
df = pd.read_csv("C:\\Users\\Affan\\Desktop\\Churn_Predictor\\data\\raw\\Telco-Customer-Churn.csv")
display(df.head())
print(df.info())

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [26]:

X = df.drop(columns=['customerID', 'Churn'])
y = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)

In [27]:
display(X.head(3))
display(y.head(3))

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15


0    0
1    0
2    1
Name: Churn, dtype: int64

In [28]:
# Splitting the dataset into trai, validation and test sets

from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, 
    test_size = 0.4,
    stratify=y,
    random_state=42)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size = 0.5,
    stratify=y_temp,
    random_state=42)

In [29]:
# check sizes
print("Train:", X_train.shape, y_train.shape)
print("Val:  ", X_val.shape, y_val.shape)
print("Test: ", X_test.shape, y_test.shape)

# check target distribution

def target_distribution(y, name):
    print(name)
    print(y.value_counts(normalize=True))
    print()

target_distribution(y_train, "Train")
target_distribution(y_val, "Validation")
target_distribution(y_test, "Test")
# target distribution is maintained across all sets => stratification worked well


Train: (4225, 19) (4225,)
Val:   (1409, 19) (1409,)
Test:  (1409, 19) (1409,)
Train
Churn
0    0.734675
1    0.265325
Name: proportion, dtype: float64

Validation
Churn
0    0.734564
1    0.265436
Name: proportion, dtype: float64

Test
Churn
0    0.734564
1    0.265436
Name: proportion, dtype: float64



In [30]:
# saving the datasets
X_train.to_csv("C:\\Users\\Affan\\Desktop\\Churn_Predictor\\data\\processed\\X_train.csv", index=False)
y_train.to_csv("C:\\Users\\Affan\\Desktop\\Churn_Predictor\\data\\processed\\y_train.csv", index=False)

X_val.to_csv("C:\\Users\\Affan\\Desktop\\Churn_Predictor\\data\\processed\\X_val.csv", index=False)
y_val.to_csv("C:\\Users\\Affan\\Desktop\\Churn_Predictor\\data\\processed\\y_val.csv", index=False)

X_test.to_csv("C:\\Users\\Affan\\Desktop\\Churn_Predictor\\data\\processed\\X_test.csv", index=False)
y_test.to_csv("C:\\Users\\Affan\\Desktop\\Churn_Predictor\\data\\processed\\y_test.csv", index=False)

In [32]:
# Defining a baseline model

from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

# Defining the baseline classifier model
baseline_clf = DummyClassifier(strategy='most_frequent', random_state=42)

# Training
baseline_clf.fit(X_train, y_train)

# Validate
y_val_pred = baseline_clf.predict(X_val)

# Evaluate
baseline_accuracy = accuracy_score(y_val, y_val_pred)

print("Baseline validation accuracy:", round(baseline_accuracy, 4))

Baseline validation accuracy: 0.7346
